In [18]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import csv
import re

# Fonction pour extraire les marques et les descriptions
def extract_phone_info(text):
    regex_patterns = {
        'Apple': r'Apple\s+iPhone\s?(?:[^\s]+)?(?:\s+Pro\s?(?:Max)?)?',
        'Samsung': r'Samsung\s+Galaxy\s?(?:[^\s]+)?',
        'Xiaomi': r'Xiaomi\s+(?:Mi\s+|Redmi\s+|13T\s+)?[^\s]+',
        'Google': r'Google\s+Pixel\s?(?:[^\s]+)?(?:\s+[^\s]+)?',
        'Huawei': r'Huawei\s+(?:P20\s+|P30\s+|P40\s+|Mate\s+)?[^\s]+',
        'Nokia': r'Nokia\s+(?:X5\s+|6\s+|7\s+|8\s+|[^\s]+)?',
        'Sony': r'Sony\s+Xperia\s?(?:[^\s]+)?(?:\s+[^\s]+)?',
        'LG': r'LG\s+(?:Wing\s+|V60\s+|G8X\s+|[^\s]+)?',
        'Motorola': r'Motorola\s+(?:Moto\s+|G84\s+|[^\s]+)?',
        'BlackBerry': r'BlackBerry\s+(?:Curve\s+|KEY2\s+|[^\s]+)?',
        'ASUS': r'ASUS\s+ROG\s+Phone\s?(?:[^\s]+)?',
        'Microsoft': r'Microsoft\s+Surface\s+Duo\s?(?:[^\s]+)?',
        'Infinix': r'Infinix\s+Note\s+40\s+Pro\+',
        'CAT': r'CAT\s?(?:[^\s]+)?',
        'Reno': r'Reno\s?(?:[^\s]+)?',
        'vivo': r'vivo\s?(?:[^\s]+)?',
        'GOPHONE': r'GOPHONE\s?(?:[^\s]+)?',
        'TECNO': r'TECNO\s?(?:Spark\s+[^\s]+)?',
        'Fairphone': r'Fairphone\s?(?:[^\s]+)?',
        'Sharp': r'Sharp\s+(?:AQUOS\s+)?[^\s]+',
        'Fujitsu': r'Fujitsu\s+(?:arrows\s+)?[^\s]+',
        'OUKITEL': r'OUKITEL\s?(?:[^\s]+)?'
    }

    phone_info = []

    for brand, pattern in regex_patterns.items():
        matches = re.findall(pattern, text, re.IGNORECASE)
        for match in matches:
            phone_info.append((brand, match))
    
    return phone_info

# Ouvrir l'URL et analyser le contenu HTML
html = urlopen('https://www.ebay.fr/b/Telephones-mobiles/9355/bn_16576651')
bs = BeautifulSoup(html.read(), 'html.parser')

# Scrapper les articles de Téléphones mobiles
info = bs.find_all('div', {'class': 's-item__info clearfix'})

# Ouvrir un fichier CSV pour écrire les données
with open('smartphones.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # Écrire les en-têtes
    writer.writerow(['Article', 'Marque', 'Version', 'Statut', 'Prix (€)', 'Prix de vente initial (€)', 'Frais de livraison (€)', 'Suivi par (personnes)', 'Vendus', 'Lien du produit'])

    # Extraire et écrire les informations de chaque article
    for item in info:
        title = item.find('h3', {'class': 's-item__title'}).get_text()
        price = item.find('span', {'class': 's-item__price'}).get_text().replace('EUR', '').strip()
        
        initial_price_element = item.find('span', {'class': 'STRIKETHROUGH'})
        if initial_price_element:
            initial_price = initial_price_element.get_text().replace('EUR', '').strip()
        else:
            initial_price = None

        text_element = item.find('span', {'class': 'NEGATIVE'})
        if text_element:
            text = text_element.get_text()
        else:
            text = ""

        link = item.find('a', {'class': 's-item__link'})['href']
        frais_livraison_element = item.find('span', {'class': 's-item__shipping s-item__logisticsCost'})
        if frais_livraison_element:
            frais_livraison = frais_livraison_element.get_text().replace('de frais de livraison', '').replace('EUR', '').strip()
        else:
            frais_livraison = "0"

        status_element = item.find('div', {'class': 's-item__subtitle'})
        if status_element:
            status = status_element.get_text()
        else:
            status = ""

        followed = 0
        vendus = 0
        if "Suivi par" in text:
            followed = ''.join([word for word in text.split() if word.isdigit()])
        elif "Il n'en reste plus qu'un" in text:
            followed = 1
        if "vendus" in text:
            vendus = ''.join([word for word in text.split() if word.isdigit()])

        # Extraire les informations sur les marques et descriptions
        extracted_info = extract_phone_info(title)
        for brand, version in extracted_info:
            # Écrire les données dans le fichier CSV
            writer.writerow([title, brand, version, status, price, initial_price, frais_livraison, followed, vendus, link])

            # Afficher les données de chaque article scrapé
            print("Article :", title)
            print("Marque :", brand)
            print("Version :", version)
            print("Statut :", status)
            print("Prix (€) :", price)
            if initial_price:
                print("Prix de vente initial (€) :", initial_price)
            print("Frais de livraison (€) :", frais_livraison)
            if followed:
                print("Suivi par (personnes) :", followed)
            if vendus:
                print("Vendus :", vendus)
            print("Lien du produit :", link)
            print("\n")


Article : BlackBerry Passport Q30 (SQW100-1) 32GB 3GB RAM Unlocked Smartphone- New Sealed
Marque : BlackBerry
Version : BlackBerry Passport
Statut : Professionnel
Prix (€) : 168,81
Frais de livraison (€) : Livraison gratuite
Vendus : 29
Lien du produit : https://www.ebay.fr/itm/394512353575?hash=item5bdac4c927:g:wkUAAOSwxStj-X66&amdata=enc%3AAQAJAAAA0Az0o1L646bvFHanK%2FOg9%2B%2FpMHnP8LZc4NbGt8KQKjelQhw3pEzl0%2BwK72bC%2BWKcWldFRc2N%2FejAhwpg0hPoL2K61MGAFm3AVcwVc2t7Ua6m5x6ClCxi7gfSZI%2FSlmt57y93fGZhtfVnmFoR0hADhlqinim7IIx9ymIV%2BptwTvF0ZUt8aMupSXD6TYzaaUPLFmWWJoYJk%2FFBOlvvnmOT1CTbncnCi8QnnuhQX4vPYphdyntp2oMALVlpPTJSUvOsx1JbLKb7ZHcXdtr1B%2BhgTrc%3D%7Ctkp%3ABFBM-N7dp4dk


Article : Unlocked Samsung Galaxy Folder G1600 Dual SIM LTE Flip SmartPhone- New Sealed
Marque : Samsung
Version : Samsung Galaxy Folder
Statut : Professionnel
Prix (€) : 82,07
Frais de livraison (€) : Livraison gratuite
Vendus : 196
Lien du produit : https://www.ebay.fr/itm/363763978897?hash=item54b205d291:g:CDYAAOSwo4x